<a href="https://colab.research.google.com/github/azzindani/03_Prompt_Engineering/blob/main/PE_EN_Qwen2_5_0_5B_Med_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Modules

In [1]:
!pip install -U -q bitsandbytes
!pip install -q optimum
!pip install -q auto-gptq

In [2]:
import torch

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  GenerationConfig,
  BitsAndBytesConfig,
  GPTQConfig
)

if torch.cuda.is_available():
  print('GPU is available!')
else:
  print('GPU is not available.')

GPU is available!


## Import Model

In [3]:
model_name = 'Qwen/Qwen2.5-0.5B'

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
'''bnb_config = BitsAndBytesConfig(
  load_in_8bit = True,
  #bnb_4bit_quant_type = 'nf4',
  #bnb_4bit_compute_dtype = torch.float16,
  #bnb_4bit_use_double_quant = True,
)

model = AutoModelForCausalLM.from_pretrained(
  model_name,
  quantization_config = bnb_config,
  trust_remote_code = True
)#.to(device) #'''

"bnb_config = BitsAndBytesConfig(\n  load_in_8bit = True,\n  #bnb_4bit_quant_type = 'nf4',\n  #bnb_4bit_compute_dtype = torch.float16,\n  #bnb_4bit_use_double_quant = True,\n)\n\nmodel = AutoModelForCausalLM.from_pretrained(\n  model_name,\n  quantization_config = bnb_config,\n  trust_remote_code = True\n)#.to(device) #"

In [6]:
model = AutoModelForCausalLM.from_pretrained(
  model_name,
  torch_dtype = torch.float16,
  trust_remote_code = True
).to(device) #'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Import Tokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Model Preview

In [8]:
def assistant(prompt):
  prompt_format = f"### Human: {prompt}\n### Assistant :"

  inputs = tokenizer(
    prompt_format,
    return_tensors = 'pt'
  ).to(device)
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = model.generate(
    **inputs,
    generation_config = generation_config
  )
  return print(tokenizer.decode(outputs[0], skip_special_tokens = True))

In [9]:
# Level 1: Basic Prompt
prompt = "What are the symptoms and treatments of diabetes?"
assistant(prompt)

### Human: What are the symptoms and treatments of diabetes?
### Assistant : Diabetes is a chronic condition that affects the body's ability to regulate blood sugar levels. It is caused by the body's inability to produce or use insulin, a hormone that helps regulate blood sugar levels. The symptoms of diabetes can vary depending on the type of diabetes and the severity of the condition. Some common symptoms of diabetes include:

1. Frequent urination
2. Increased thirst
3. Fatigue
4. Numbness or tingling in the hands or feet
5. Blurred vision
6. Unexplained weight loss
7. Increased hunger
8. Frequent infections

The treatments for diabetes depend on the type of diabetes and the severity of the condition. For type 1 diabetes, insulin injections are the main treatment. For type 2 diabetes, lifestyle changes, such as diet and exercise, are the main treatments. In some cases, medications may be prescribed to help control blood sugar levels. It is important to consult with a healthcare prof

In [10]:
# Level 2: Adding Detail
prompt = "Briefly describe the symptoms and treatments of diabetes."
assistant(prompt)

### Human: Briefly describe the symptoms and treatments of diabetes.
### Assistant : Diabetes is a chronic condition that affects the body's ability to regulate blood sugar levels. It is caused by the body's inability to produce or use insulin, a hormone that helps regulate blood sugar levels. The symptoms of diabetes can vary depending on the type of diabetes and the severity of the condition. Some common symptoms include frequent urination, thirst, fatigue, weight loss, and blurred vision. The treatments for diabetes include lifestyle changes, medication, and insulin therapy. Lifestyle changes such as diet, exercise, and weight loss can help manage the symptoms of diabetes. Medications such as oral hypoglycemic agents and insulin can help regulate blood sugar levels. Insulin therapy is recommended for people with type 1 diabetes.


In [11]:
# Level 3: Specify Information Type and Audience
prompt = "Explain the common symptoms and treatments of diabetes to a general audience."
assistant(prompt)

### Human: Explain the common symptoms and treatments of diabetes to a general audience.
### Assistant : Diabetes is a chronic condition that affects the body's ability to regulate blood sugar levels. It is caused by the body's inability to produce or use insulin, a hormone that helps regulate blood sugar levels. The most common symptoms of diabetes include frequent urination, thirst, fatigue, weight loss, and blurred vision. The treatment of diabetes includes lifestyle changes, such as diet and exercise, and medication to regulate blood sugar levels.


In [12]:
# Level 4: Request for Structured Response
prompt = "List the main symptoms of diabetes, then outline the standard treatments."
assistant(prompt)

### Human: List the main symptoms of diabetes, then outline the standard treatments.
### Assistant : Diabetes is a chronic disease that affects the body's ability to regulate blood sugar levels. The main symptoms of diabetes include frequent urination, thirst, fatigue, weight loss, and blurred vision. The standard treatments for diabetes include diet changes, exercise, and medication.


In [13]:
# Level 5: Differentiate by Type
prompt = "List the symptoms and treatments for both Type 1 and Type 2 diabetes."
assistant(prompt)

### Human: List the symptoms and treatments for both Type 1 and Type 2 diabetes.
### Assistant : Type 1 diabetes is a chronic condition that affects the body's ability to produce insulin, leading to high blood sugar levels. It is typically diagnosed in childhood or adolescence and can be managed with insulin therapy. Type 2 diabetes, on the other hand, is a chronic condition that affects the body's ability to use insulin effectively. It is typically diagnosed in adulthood and can be managed with lifestyle changes and medication.


In [14]:
# Level 6: Target a Professional Audience
prompt = "Describe the symptoms, diagnostic criteria, and treatment options for Type 1 and Type 2 diabetes, focusing on what healthcare providers should know."
assistant(prompt)

### Human: Describe the symptoms, diagnostic criteria, and treatment options for Type 1 and Type 2 diabetes, focusing on what healthcare providers should know.
### Assistant : Type 1 and Type 2 diabetes are both chronic conditions that require medical attention. They are caused by the body's inability to produce enough insulin or respond to insulin effectively. The symptoms of both types of diabetes can vary, but they often include frequent urination, excessive thirst, fatigue, and weight loss. The diagnostic criteria for Type 1 diabetes include symptoms such as sudden weight loss, increased thirst, frequent urination, and blurred vision. For Type 2 diabetes, the diagnostic criteria include symptoms such as increased thirst, frequent urination, weight loss, and fatigue. Treatment options for both types of diabetes include lifestyle changes, medication, and insulin therapy.


In [15]:
# Level 7: Clinical Detail with Disease Stages
prompt = "Explain the symptoms, diagnostic markers, and treatments for diabetes, distinguishing between prediabetes, Type 1, and Type 2 diabetes. Include early signs, progression, and advanced stages."
assistant(prompt)

### Human: Explain the symptoms, diagnostic markers, and treatments for diabetes, distinguishing between prediabetes, Type 1, and Type 2 diabetes. Include early signs, progression, and advanced stages.
### Assistant : Diabetes is a chronic condition that affects the body's ability to regulate blood sugar levels. It is a group of metabolic disorders that result in high blood sugar levels, which can lead to various complications if left untreated. The main symptoms of diabetes include frequent urination, thirst, fatigue, weight loss, and blurred vision. The diagnostic markers for diabetes include fasting blood glucose levels, blood pressure, and cholesterol levels. The early signs of diabetes include frequent urination, thirst, and fatigue. As the condition progresses, the symptoms may become more severe, and the body may develop complications such as kidney disease, nerve damage, and blindness. The advanced stages of diabetes include diabetic ketoacidosis, which is a life-threatening co

In [16]:
# Level 8: Add Nuance with Treatment Personalization
prompt = "Describe the symptoms, diagnostic markers, and treatment options for diabetes, with a focus on Type 1 and Type 2. Include how treatments may vary based on patient age, co-existing conditions, and lifestyle factors."
assistant(prompt)

### Human: Describe the symptoms, diagnostic markers, and treatment options for diabetes, with a focus on Type 1 and Type 2. Include how treatments may vary based on patient age, co-existing conditions, and lifestyle factors.
### Assistant : Diabetes is a chronic condition that affects millions of people worldwide. It is a group of metabolic disorders characterized by high blood sugar levels, which can lead to various complications if not managed properly. Type 1 diabetes is an autoimmune disease where the body's immune system attacks and destroys the insulin-producing cells in the pancreas, leading to a lack of insulin. Type 2 diabetes, on the other hand, is a more common form of the disease where the body's cells become resistant to insulin, leading to high blood sugar levels. The symptoms of diabetes can vary depending on the type and severity of the condition, but common symptoms include frequent urination, thirst, fatigue, and unexplained weight loss. Diagnosing diabetes involves 

In [17]:
# Level 9: Consider Risks, Complications, and Treatment Impacts
prompt = "Provide a comprehensive overview of diabetes, covering symptoms, diagnostic criteria, and treatment options for both Type 1 and Type 2 diabetes. Explain how risks and complications, such as neuropathy and cardiovascular disease, may impact treatment decisions. Include possible side effects and patient adherence concerns for each treatment."
assistant(prompt)

### Human: Provide a comprehensive overview of diabetes, covering symptoms, diagnostic criteria, and treatment options for both Type 1 and Type 2 diabetes. Explain how risks and complications, such as neuropathy and cardiovascular disease, may impact treatment decisions. Include possible side effects and patient adherence concerns for each treatment.
### Assistant : 
Diabetes is a chronic condition that affects the body's ability to regulate blood sugar levels. It is a group of disorders that affect the way the body uses glucose (sugar) for energy. There are two main types of diabetes: Type 1 and Type 2.

#### Symptoms of Diabetes
- **Frequent urination**
- **Increased thirst**
- **Unexplained weight loss**
- **Fatigue**
- **Numbness or tingling in the hands or feet**
- **Blurred vision**
- **Unexplained weight gain**
- **Increased risk of heart disease, stroke, and kidney disease**

#### Diagnostic Criteria
- **Fasting Blood Sugar (FBS) Level**: 126 mg/dL or higher
- **Hemoglobin A1c 

In [18]:
# Level 10: Advanced Detail for Clinical Research or Specialist Training
prompt = "Provide an in-depth, evidence-based analysis of the symptoms, diagnostic criteria, and treatment approaches for diabetes, including both Type 1 and Type 2. Discuss the pathophysiology, biomarkers, and clinical guidelines, as well as how complications like neuropathy, retinopathy, and cardiovascular disease alter treatment. Include recent clinical research findings, new pharmaceutical options, and the role of lifestyle interventions in managing long-term patient outcomes."
assistant(prompt)

### Human: Provide an in-depth, evidence-based analysis of the symptoms, diagnostic criteria, and treatment approaches for diabetes, including both Type 1 and Type 2. Discuss the pathophysiology, biomarkers, and clinical guidelines, as well as how complications like neuropathy, retinopathy, and cardiovascular disease alter treatment. Include recent clinical research findings, new pharmaceutical options, and the role of lifestyle interventions in managing long-term patient outcomes.
### Assistant : Provide an in-depth, evidence-based analysis of the symptoms, diagnostic criteria, and treatment approaches for diabetes, including both Type 1 and Type 2. Discuss the pathophysiology, biomarkers, and clinical guidelines, as well as how complications like neuropathy, retinopathy, and cardiovascular disease alter treatment. Include recent clinical research findings, new pharmaceutical options, and the role of lifestyle interventions in managing long-term patient outcomes.

### Diagnosis and Tr